In [ ]:
#| default_exp block

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import numpy as np, re, inspect
from typing import Optional, Dict
from transformers import AutoTokenizer, BatchEncoding

from fastcore.meta import *

from xcai.data import *
from xcai.transform import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Config

In [ ]:
#| export
PARAM = {
    'cols': ['identifier', 'input_text'],
    'use_tokz': True,
    'tokz': 'bert-base-cased',
    'fld': 'input_text',
    'max_len': 32,
    'pad_side': 'right',
    'inp': 'data',
    'targ': 'lbl2data',
    'ptr': 'lbl2data_data2ptr',
    'drop': True,
    'ret_t': True,
    'in_place': True,
    'collapse': True,
    'device': 'cpu',
    'tfm': 'xc',
}

In [ ]:
#| export
TRAIN_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
DATA_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
TRAIN_META_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt'
            },
        },
    },
    'parameters': PARAM,
}

In [ ]:
#| export
DATA_META_CFG = {
    'path': {
        'train': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/trn_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_train.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_trn_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt'
            },
        },
        'test': {
            'data_lbl': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/tst_X_Y.txt',
            'data_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/test.raw.txt',
            'lbl_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/label.raw.txt',
            'data_lbl_filterer': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/filter_labels_test.txt',
            'hlk_meta': {
                'prefix': 'hlk',
                'data_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_tst_X_Y.txt',
                'lbl_meta': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/hyper_link_lbl_X_Y.txt',
                'meta_info': '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/hyper_link.raw.txt',
            },
        },
    },
    'parameters': PARAM,
}

## Block

In [ ]:
#| export
CFGS = {'train':TRAIN_CFG, 'data':DATA_CFG, 'train_meta':TRAIN_META_CFG, 'data_meta':DATA_META_CFG, }
TFMS = {'xc': [XCPadFeatTfm, AlignInputIdsTfm], 'ng': [XCPadFeatTfm, AlignInputIdsTfm, NGSampleFeatTfm, NGMapper],}

In [ ]:
#| export
class XCBlock:

    @delegates(XCDataBlock.from_cfg)
    @classmethod
    def from_cfg(cls, cfg:str, bsz:Optional[int]=10, **kwargs):
        if cfg in CFGS: cfg = CFGS[cfg]
        else: raise ValueError(f"Invalid `cfg` value {cfg}, it should be one of ['train', 'data', 'train_meta', 'data_meta']")
        for k in cfg['parameters']: 
            if k in kwargs and kwargs[k] is not None: cfg['parameters'][k]=kwargs.pop(k)
        tokz = AutoTokenizer.from_pretrained(cfg['parameters']['tokz'])
        cfg['parameters']['sep_tok'], cfg['parameters']['pad_tok'], cfg['parameters']['batch_size'] = tokz.sep_token_id, tokz.pad_token_id, bsz
        collator = XCCollator(TfmPipeline([o(**cfg['parameters']) for o in TFMS[cfg['parameters']['tfm']]]))
        return XCDataBlock.from_cfg(cfg, collate_fn=collator, **kwargs)


### Example

In [ ]:
block = XCBlock.from_cfg('train', tfm='ng', tokz='bert-base-uncased')

In [ ]:
b = block.train.one_batch()

In [ ]:
b.keys()

dict_keys(['lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask', 'lbl2data_idx_smp', 'lbl2data_identifier_smp', 'lbl2data_input_text_smp', 'lbl2data_input_ids_smp', 'lbl2data_token_type_ids_smp', 'lbl2data_attention_mask_smp', 'lbl2data_data2ptr_smp'])

In [ ]:
import torch
for k,v in b.items():
    if isinstance(v, torch.Tensor): print(k,':', v.shape)
    else: print(k,':',len(v))

lbl2data_idx : torch.Size([21])
lbl2data_identifier : 21
lbl2data_input_text : 21
lbl2data_input_ids : torch.Size([10, 14])
lbl2data_token_type_ids : torch.Size([10, 14])
lbl2data_attention_mask : torch.Size([10, 14])
lbl2data_data2ptr : torch.Size([10])
data_identifier : 10
data_input_text : 10
data_input_ids : torch.Size([10, 14])
data_token_type_ids : torch.Size([10, 14])
data_attention_mask : torch.Size([10, 14])
lbl2data_idx_smp : torch.Size([10])
lbl2data_identifier_smp : 10
lbl2data_input_text_smp : 10
lbl2data_input_ids_smp : torch.Size([10, 14])
lbl2data_token_type_ids_smp : torch.Size([10, 14])
lbl2data_attention_mask_smp : torch.Size([10, 14])
lbl2data_data2ptr_smp : torch.Size([10])


In [ ]:
b = block.train.dset.one_batch()

In [ ]:
b

[{'data_identifier': 'Ferron_Formation',
  'data_input_text': 'Ferron Formation',
  'data_input_ids': [101, 10768, 18933, 2078, 4195, 102],
  'data_token_type_ids': [0, 0, 0, 0, 0, 0],
  'data_attention_mask': [1, 1, 1, 1, 1, 1],
  'lbl2data_idx': [246699, 246701],
  'lbl2data_identifier': ['List_of_fossiliferous_stratigraphic_units_in_Utah',
   'Paleontology_in_Utah'],
  'lbl2data_input_text': ['List of fossiliferous stratigraphic units in Utah',
   'Paleontology in Utah'],
  'lbl2data_input_ids': [[101,
    2862,
    1997,
    10725,
    23930,
    2358,
    8609,
    8004,
    20721,
    3197,
    1999,
    6646,
    102],
   [101, 5122, 12162, 6779, 1999, 6646, 102]],
  'lbl2data_token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0]],
  'lbl2data_attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1]]},
 {'data_identifier': 'Jet_(lignite)',
  'data_input_text': 'Jet (lignite)',
  'data_input_ids': [101, 6892, 1006, 5622, 29

## Batch

In [ ]:
#| export
def prepare_batch(m, b, m_args=None):
    m_kwargs = inspect.signature(m.forward).parameters
    return BatchEncoding({k:v for k,v in b.items() if k in m_kwargs or (m_args is not None and k in m_args)})